In [ ]:
# 1-1. 필수 라이브러리 설치 (Colab 셀에서 실행)
!pip install sentence-transformers pandas tqdm

# 1-2. 파일 업로드
# 좌측 폴더 아이콘을 클릭하여 'java_dpo.json', 'python_dpo.json', 'java_mega_vuln.json' 파일을 업로드합니다.

import json
import pandas as pd
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm  # Colab 환경에서 tqdm.auto 사용
import os
import hashlib  # 중복 제거를 위해 추가
from typing import List
import sys
from pathlib import Path

# --- 설정 변수 ---
MODEL_NAME = "sentence-transformers/multi-qa-distilbert-cos-v1"
OUTPUT_FILENAME = "secure_coding_embeddings.csv"
CODE_VECTOR_DIMENSION = 768  # CodeBERT 모델의 예상 차원

FILE_PATHS = [
    "java_dpo.json",
    "python_dpo.json",
    "java_mega_vuln.json"
]


# --- 1. 데이터 로드, 중복 제거 및 RAG 청크 생성 ---
def load_and_preprocess_data(file_paths: list) -> pd.DataFrame:
    """JSON 파일을 로드하고 각 레코드에 정확한 출처를 매핑하며, 중복을 제거합니다."""
    all_processed_data = []
    unique_hashes = set()

    existing_files = [f for f in file_paths if os.path.exists(f)]

    if not existing_files:
        print("🚨 오류: 파일을 찾을 수 없습니다. Colab에 JSON 파일을 업로드했는지 확인하세요.")
        return pd.DataFrame()

    print(f"✅ 다음 파일들을 로드합니다: {existing_files}")

    for file_path_str in existing_files:
        file_name = os.path.basename(file_path_str)  # ✨ 정확한 파일명 추출

        try:
            with open(file_path_str, 'r', encoding='utf-8') as f:
                file_data = json.load(f)
        except Exception as e:
            print(f"경고: 파일 '{file_path_str}' 로드/파싱 실패: {e}. 건너뜁니다.")
            continue

        for item in file_data:
            # --- 핵심: 필드 통합 매핑 ---
            description = item.get('vulnerability_description', item.get('vulnerability_summary', 'N/A'))
            safe_code = item.get('safe_code', item.get('patched_code', 'N/A'))
            vulnerable_code = item.get('vulnerable_code', 'N/A')

            # ✨ 중복 제거 로직: 취약 코드와 안전 코드를 합쳐 해시 생성
            hash_input = (vulnerable_code + safe_code).encode('utf-8')
            unique_hash = hashlib.sha256(hash_input).hexdigest()

            if unique_hash in unique_hashes:
                continue  # 중복 레코드 건너뛰기
            unique_hashes.add(unique_hash)

            # CodeBERT 임베딩에 최적화된 결합 텍스트
            context = (
                f"CWE-ID: {item.get('cwe_id', 'N/A')}\n"
                f"취약점 설명: {description}\n"
                f"취약한 코드:\n{vulnerable_code}\n"
                f"안전한 코드:\n{safe_code}"
            )

            # Qdrant Payload 메타데이터
            payload_dict = {
                "cwe_id": item.get('cwe_id', 'N/A'),
                "description": description,
                "safe_code": safe_code,
                "vulnerable_code": vulnerable_code,
                "abstract_vulnerable_code": item.get('abstract_vulnerable_code', 'N/A'),
                "language": item.get('language', file_name.split('_')[0].lower()),  # 언어 필드 추가 (파일명 기반)
                "source_file": file_name,  # ✨ 정확한 원본 파일 이름 기록
            }

            all_processed_data.append({
                "text": context,
                "payload_json": json.dumps(payload_dict, ensure_ascii=False),
                "unique_hash": unique_hash,
                "source_file_raw": file_name  # ✨ 검증을 위해 원본 파일명 임시 저장
            })

    df = pd.DataFrame(all_processed_data)

    # 🛑 데이터셋 검증 로직 호출 (출처 정확성 확인)
    if not df.empty:
        validate_dataset_integrity(df, existing_files)

    print(f"초기 로드된 총 레코드 수 (중복 제거 전): {len(df)}")

    # 최종 ID 재할당 및 정리
    df = df.drop(columns=['unique_hash'])  # 중복 제거는 이미 완료, 해시 컬럼 삭제
    df = df.drop_duplicates(subset=['text'])  # 혹시 모를 텍스트 중복 추가 제거
    df['id'] = df.index.astype(int) + 1  # Qdrant Point ID용 정수 ID 할당
    df = df.drop(columns=['source_file_raw'])  # 검증용 컬럼 삭제

    print(f"✅ 최종 레코드 수: {len(df)}")
    return df


def validate_dataset_integrity(df: pd.DataFrame, expected_files: List[str]):
    """
    병합된 DataFrame의 'source_file' 메타데이터가 예상되는 모든 원본 파일을
    정확히 참조하고 있는지 검증합니다.
    """
    print("\n--- 데이터셋 출처 검증 시작 ---")

    # 1. DataFrame에 기록된 모든 고유한 출처 파일명 추출
    actual_source_files_in_df = set(df['source_file_raw'].unique())

    # 2. 예상되는 파일 목록을 문자열 이름으로 변환
    expected_file_names = {os.path.basename(f) for f in expected_files}

    # 3. 누락된 파일 검사
    missing_files = expected_file_names - actual_source_files_in_df

    # 4. 예상치 못한 파일 검사
    unexpected_files = actual_source_files_in_df - expected_file_names

    if missing_files:
        print(f"❌ 경고: 예상된 파일 중 데이터에 포함되지 않은 파일: {missing_files}")

    if unexpected_files:
        print(f"❌ 오류: 로드 목록에 없는데 데이터에 포함된 출처 파일: {unexpected_files}")
        # 이 오류는 심각하므로 종료를 권장하지만, Colab에서는 경고로 처리하고 다음 셀에서 임베딩하도록 함.

    if not missing_files and not unexpected_files:
        print(f"✅ 출처 검증 성공: {len(actual_source_files_in_df)}개 파일의 출처가 정확히 기록됨.")

    print("----------------------------")


def generate_and_save_embeddings(df: pd.DataFrame, model_name: str, output_path: str):
    """데이터를 임베딩하고 벡터와 메타데이터를 CSV 파일로 저장합니다."""

    if df.empty:
        print("데이터프레임이 비어 있어 임베딩을 진행할 수 없습니다.")
        return

    # Colab GPU 사용 설정: CUDA가 사용 가능하면 GPU를 사용합니다.
    device = "cuda" if os.environ.get('COLAB_GPU', 'False') == 'True' else "cpu"
    print(f"Loading embedding model: {model_name} on device: {device}...")

    model = SentenceTransformer(model_name, device=device)
    vector_dim = model.get_sentence_embedding_dimension()

    if vector_dim != CODE_VECTOR_DIMENSION:
        print(f"⚠️ 경고: 설정 차원({CODE_VECTOR_DIMENSION})과 모델 차원({vector_dim}) 불일치. config_db 업데이트 필요.")

    print(f"Generating embeddings for {len(df)} data points...")
    vectors = model.encode(
        df['text'].tolist(),
        show_progress_bar=True,
        normalize_embeddings=True,
        batch_size=128  # GPU 환경에 최적화된 배치 크기
    )

    # 벡터를 문자열로 변환하여 CSV에 저장
    df['vector'] = [vec.tolist() for vec in vectors]

    # 최종 CSV 저장: ID, 벡터, 메타데이터 JSON
    final_df = df[['id', 'vector', 'payload_json']]
    final_df.to_csv(output_path, index=False)

    print(f"\n✅ 임베딩 생성 및 파일 저장 완료: '{output_path}'")
    print("Colab 좌측 메뉴에서 이 파일을 다운로드하여 다음 단계에서 사용하세요.")
    print(f"총 데이터 포인트: {len(df)}, 벡터 차원: {vector_dim}")


# 실행
df_final = load_and_preprocess_data(FILE_PATHS)
generate_and_save_embeddings(df_final, MODEL_NAME, OUTPUT_FILENAME)